<a href="https://colab.research.google.com/github/anjelammcgraw/LlamaIndex-RAG/blob/main/12_Creating_a_LlamaIndex_RAG_Pipeline_with_NL2SQL_and_Metadata_Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating a LlamaIndex RAG Pipeline with NL2SQL and Metadata Filtering!

We'll be putting together a system for querying both qualitative and quantitative data using LlamaIndex.


This is only relevant when running the code in a Jupyter Notebook.

In [ ]:
import nest_asyncio

nest_asyncio.apply()

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## Load Dependencies

In [ ]:
!pip install -qU llama-index openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -qU wandb llama-index-callbacks-wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.8/258.8 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.1 MB/s eta 0:00:00


In [ ]:
!pip install -qU wikipedia llama-index-readers-wikipedia

  Preparing metadata (setup.py) ... done


In [ ]:
!pip install -qU chromadb llama-index-vector-stores-chroma

In [ ]:
!pip install -q -U sqlalchemy pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 31.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.23.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.1 which is incompatible.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.2.1 which is incompatible.


In [ ]:
!pip install -U -q tiktoken==0.4.0 sentence-transformers==2.2.2 pydantic==1.10.11

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 18.7 MB/s eta 0:00:00


## Set Env Variables and Set Up WandB Callback

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key: ")

OpenAI API Key: ··········


In [ ]:
os.environ["WANDB_API_KEY"] = getpass.getpass("WandB API Key: ")

WandB API Key: ··········


In [ ]:
import llama_index
from llama_index.core import set_global_handler

set_global_handler("wandb", run_args={"project": "aie1-llama-index-demo"})
wandb_callback = llama_index.core.global_handler

wandb: Streaming LlamaIndex events to W&B at https://wandb.ai/anjelammcgraw/aie1-llama-index-demo/runs/odgsscl0
wandb: `WandbCallbackHandler` is currently in beta.
wandb: Please report any issues to https://github.com/wandb/wandb/issues with the tag `llamaindex`.


### Settings


In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

## Semantic RAG Pipeline with Metadata Filtering



### Data Collection


In [ ]:
from llama_index.readers.wikipedia import WikipediaReader

movie_list = ["Dune (2021 film)", "Dune: Part Two"]

wiki_docs = WikipediaReader().load_data(pages=movie_list, auto_suggest=False)

### Initializing our VectorStoreIndex with ChromaDB


In [ ]:
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.create_collection("dune-v0")

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents([], storage_context=storage_context)

wandb: Logged trace tree to W&B.


### Node Construction


In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.extractors import TitleExtractor

pipeline = IngestionPipeline(transformations=[TokenTextSplitter()])

for movie, wiki_doc in zip(movie_list, wiki_docs):
    nodes = pipeline.run(documents=wiki_docs)
    for node in nodes:
        node.metadata = {"title" : movie}
    index.insert_nodes(nodes)

Insert of existing embedding ID: b1b20e6a-932a-41fa-92ba-519d10cd117d


Insert of existing embedding ID: a92f3156-a80c-432e-b97e-26d0e8b59c40


Insert of existing embedding ID: a1113b93-166b-4cb4-be28-043a986d4af6


Insert of existing embedding ID: 5a200b42-2fee-407e-8ed8-e2b98e3aea58


Insert of existing embedding ID: c90089bc-ebc9-4387-b274-0227a8c825e6


...

Add of existing embedding ID: 6e125eda-dc28-4b71-9824-315ea0f5ba68


Add of existing embedding ID: aec9fd86-a2df-4798-8aac-09eaaa2da57a


Add of existing embedding ID: 129d4576-8aa1-4041-b40c-378f15a3ef69


Add of existing embedding ID: 0cff4de7-52c1-4b70-bf8d-57e513d820f9


Add of existing embedding ID: a3714866-5281-48d3-83de-88fa5a152901


In [ ]:
nodes[0].metadata

{'title': 'Dune: Part Two'}

### Persisting and Loading Stored Index with Weights and Biases


In [ ]:
wandb_callback.persist_index(index, index_name="dune-index-chromadb")

wandb: Adding directory to artifact (/content/wandb/run-20240312_222507-odgsscl0/files/storage)... Done. 0.0s


In [ ]:
from llama_index.core import load_index_from_storage

storage_context = wandb_callback.load_storage_context(
    artifact_url="anjelammcgraw/aie1-llama-index-demo/dune-index-chromadb:v1"
)

wandb:   4 of 4 files downloaded.  


### Auto Retriever Functional Tool



In [ ]:
from llama_index.core.tools import FunctionTool
from llama_index.core.vector_stores.types import (
    VectorStoreInfo,
    MetadataInfo,
    ExactMatchFilter,
    MetadataFilters,
)
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

from typing import List, Tuple, Any
from pydantic import BaseModel, Field

top_k = 3

vector_store_info = VectorStoreInfo(
    content_info="semantic information about movies",
    metadata_info=[MetadataInfo(
        name="title",
        type="str",
        description="title of the movie, one of ['Dune (2021 film)', 'Dune: Part 2']",
    )]
)

In [ ]:
class AutoRetrieveModel(BaseModel):
    query: str = Field(..., description="natural language query string")
    filter_key_list: List[str] = Field(
        ..., description="List of metadata filter field names"
    )
    filter_value_list: List[str] = Field(
        ...,
        description=(
            "List of metadata filter field values (corresponding to names specified in filter_key_list)"
        )
    )

In [ ]:
def auto_retrieve_fn(
    query: str, filter_key_list: List[str], filter_value_list: List[str]
):
    """Auto retrieval function.

    Performs auto-retrieval from a vector database, and then applies a set of filters.

    """
    query = query or "Query"

    exact_match_filters = [
        ExactMatchFilter(key=k, value=v)
        for k, v in zip(filter_key_list, filter_value_list)
    ]
    retriever = VectorIndexRetriever(
        index, filters=MetadataFilters(filters=exact_match_filters), top_k=top_k
    )
    query_engine = RetrieverQueryEngine.from_args(retriever)

    response = query_engine.query(query)
    return str(response)

In [ ]:
description = f"""\
Use this tool to look up semantic information about films.
The vector database schema is given below:
{vector_store_info.json()}
"""

auto_retrieve_tool = FunctionTool.from_defaults(
    fn=auto_retrieve_fn,
    name="semantic-film-info",
    description=description,
    fn_schema=AutoRetrieveModel
)

####❓ Question #3:

Is the text in the description of our `FunctionTool` important or not? Please explain your answer.

#### ✅ Answer:

By providing a description, the tool sets expectations of what it can and cannot do and offers guidance on how to formulate requests or commands to the tool. Descriptions can also help maintan consister and standardization across the interface, given that there is a system with multiple tools.

In [ ]:
from llama_index.agent.openai import OpenAIAgent

agent = OpenAIAgent.from_tools(
    tools=[auto_retrieve_tool],
    verbose=True,
)

In [ ]:
response = agent.chat("Who starred in the 2021 film?")
print(str(response))

Added user message to memory: Who starred in the 2021 film?
=== Calling Function ===
Calling function: semantic-film-info with args: {"query":"cast of Dune (2021 film)","filter_key_list":["title"],"filter_value_list":["Dune (2021 film)"]}
Got output: The cast of "Dune (2021 film)" includes Timothée Chalamet as Paul "Muad'Dib" Atreides, Zendaya as Chani, Rebecca Ferguson as Lady Jessica, Josh Brolin as Gurney Halleck, Austin Butler as Feyd-Rautha Harkonnen, Florence Pugh as Princess Irulan, Dave Bautista as Glossu Rabban Harkonnen, Christopher Walken as Shaddam IV, Léa Seydoux as Lady Margot Fenring, Souheila Yacoub as Shishakli, Stellan Skarsgård as Baron Vladimir Harkonnen, Charlotte Rampling as Gaius Helen Mohiam, and Javier Bardem as Stilgar.



wandb: Logged trace tree to W&B.


The cast of the 2021 film "Dune" includes Timothée Chalamet, Zendaya, Rebecca Ferguson, Josh Brolin, Austin Butler, Florence Pugh, Dave Bautista, Christopher Walken, Léa Seydoux, Souheila Yacoub, Stellan Skarsgård, Charlotte Rampling, and Javier Bardem.


## Quantitative RAG Pipeline with NL2SQL Tooling



1. Read in our `.csv` files into `pd.DataFrame` objects
2. Create an in-memory `sqlite` powered `sqlalchemy` engine
3. Cast our `pd.DataFrame` objects to the SQL engine
4. Create an `SQLDatabase` object through LlamaIndex
5. Use that to create a `QueryEngineTool` that we can interact with through the `NLSQLTableQueryEngine`!

In [ ]:
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/dune1.csv

--2024-03-12 22:26:22--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/dune1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 133391 (130K) [text/plain]
Saving to: ‘dune1.csv’

dune1.csv           100%[===================>] 130.26K  --.-KB/s    in 0.02s   

2024-03-12 22:26:22 (5.97 MB/s) - ‘dune1.csv’ saved [133391/133391]



In [ ]:
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/dune2.csv

--2024-03-12 22:26:22--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/dune2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 111843 (109K) [text/plain]
Saving to: ‘dune2.csv’

dune2.csv           100%[===================>] 109.22K  --.-KB/s    in 0.02s   

2024-03-12 22:26:22 (5.14 MB/s) - ‘dune2.csv’ saved [111843/111843]



#### Read `.csv` Into Pandas

In [ ]:
import pandas as pd

dune1_df = pd.read_csv("./dune1.csv")
dune2_df = pd.read_csv("./dune2.csv")

#### Create SQLAlchemy engine with SQLite

In [ ]:
from sqlalchemy import create_engine

engine = create_engine("sqlite+pysqlite:///:memory:")

#### Convert `pd.DataFrame` to SQL tables

In [ ]:
dune1_df.to_sql(
    "Dune (2021 film)",
    engine
)

274

In [ ]:
dune2_df.to_sql(
    "Dune: Part 2",
    engine
)

175

#### Construct a `SQLDatabase` index


In [ ]:
from llama_index.core import SQLDatabase

sql_database = SQLDatabase(
    engine=engine,
    include_tables=["Dune (2021 film)", "Dune: Part 2"]
)

#### Create the NLSQLTableQueryEngine interface for all added SQL tables


In [ ]:
from llama_index.core.indices.struct_store.sql_query import NLSQLTableQueryEngine

sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["Dune (2021 film)", "Dune: Part 2"],
)

#### Wrap It All Up in a `QueryEngineTool`
EXAMPLE:
```
Useful for translating a natural language query into a SQL query over a table containing:
John Wick 1, containing information related to reviews of the first John Wick movie, called John Wick
John Wick 2, containing information related to reviews of the second John Wick movie, called John Wick: Chapter 2
John Wick 3, containing information related to reviews of the third John Wick movie, called John Wick: Chatper 3 - Parabellum
John Wick 4, containing information related to reviews of the fourth John Wick movie, called John Wick: Chatper 4
```



In [ ]:
DESCRIPTION = """\
This tables are for storing, analyzing, organizing reviews and ratings of the 2021 science fiction movie "Dune 1 &, 2" directed by Denis Villeneuve. This is for translating a natural language query in a SQL query over a table including several columns that detail the aspects of each review for "Dune 1 (2021 film)", "Dune: Part 2" which are: Review_Date, Author, Rating, Review_Title, Review, Review_Url"""


In [ ]:
from llama_index.core.tools.query_engine import QueryEngineTool

sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    name="sql-query",
    description=DESCRIPTION,
)

In [ ]:
agent = OpenAIAgent.from_tools(
    tools=[sql_tool],
    verbose=True
)

In [ ]:
response = agent.chat("What is the average rating of the 2nd Dune movie?")

Added user message to memory: What is the average rating of the 2nd Dune movie?
=== Calling Function ===
Calling function: sql-query with args: {"input":"SELECT AVG(Rating) AS Average_Rating FROM Movie_Reviews WHERE Review_Title = 'Dune: Part 2'"}
Got output: The average rating for "Dune: Part 2" is not available in the database.



wandb: Logged trace tree to W&B.


In [ ]:
print(str(response))

I'm sorry, but the average rating for "Dune: Part 2" is not available in the database.


### Combined RAG Pipeline



In [ ]:
dune_agent = OpenAIAgent.from_tools(
    tools=[auto_retrieve_tool, sql_tool],
    verbose=True
)

In [ ]:
response = dune_agent.chat("What is the lowest rating of the 1st film?")

Added user message to memory: What is the lowest rating of the 1st film?
=== Calling Function ===
Calling function: sql-query with args: {"input":"SELECT MIN(Rating) AS Lowest_Rating FROM film_reviews WHERE Title = 'Dune (2021 film)'"}
Got output: The lowest rating for the film "Dune (2021 film)" is 1.0.



wandb: Logged trace tree to W&B.


In [ ]:
print(str(response))

The lowest rating for the 1st film "Dune (2021 film)" is 1.0.


In [ ]:
response = dune_agent.chat("What planet does the 1st Dune movie take place on?")

Added user message to memory: What planet does the 1st Dune movie take place on?
=== Calling Function ===
Calling function: semantic-film-info with args: {"query":"planet where the 1st Dune movie takes place","filter_key_list":["title"],"filter_value_list":["Dune (2021 film)"]}
Got output: Arrakis



wandb: Logged trace tree to W&B.


In [ ]:
print(str(response))

The 1st Dune movie "Dune (2021 film)" takes place on the planet Arrakis.


In [ ]:
response = dune_agent.chat("Calculate the average review of each movie - and then discuss how the average review changed over time.")

Added user message to memory: Calculate the average review of each movie - and then discuss how the average review changed over time.
=== Calling Function ===
Calling function: sql-query with args: {"input": "SELECT Title, AVG(Rating) AS Average_Review FROM film_reviews GROUP BY Title"}
Got output: It seems there was an error in the SQL query. The correct syntax for calculating the average rating for each film in the film_reviews table should be: SELECT Title, AVG(Rating) AS Average_Review FROM film_reviews GROUP BY Title. This query will provide you with the average review rating for each film title in the database.

=== Calling Function ===
Calling function: sql-query with args: {"input": "SELECT Review_Date, AVG(Rating) AS Average_Review FROM film_reviews GROUP BY Review_Date ORDER BY Review_Date"}
Got output: There seems to be an error in the SQL query provided. It appears that the statement is invalid. Please review the syntax and try running the query again.

=== Calling Function

wandb: Logged trace tree to W&B.


In [ ]:
print(str(response))

I encountered an error while trying to calculate the average review for each movie and over time. Let me correct the queries and try again.


####❓ Question #4:

How can you verify which tool was used for which query?

#### ✅ Answer:
We use the **'verbose=True'** parameter in the "OpenAIAgent.from_tools()" method. With that, it makes the agent log and the output gives us additional details about its ops, including specifying which tool it decides to use for the query.   

**Query:** "What is the lowest rating of the 1st film?" We used the **'sql_tool'** for this query

**Reason:** The query asks for specific data, we use the **'sql_tool'**to fetch the lowest rating for the first film.

**Query:** "What planet does the 1st Dune movie take place on?" We used the **'auto_retrieve_tool'** for this query.

**Reason:** The **'auto-retrieve'tool** is a knowledge-based tool within the agent. It's designed for getting facts from a pre-defined knowledge based/from the internet.

**Query:** "Calculate the average review of each move - and then discuss how the average review changed over time." We used the **'sql_tool'** for this query.

**Reason:** In this query, the **'sql_tool'** processes the numerical data from the reviews stored in database. Then, discussing how the review changed over time involves a tool that can interpret results and provide insights/a summary. The **'sql_tool'** has a component that can handle natural language understanding generation.

In [ ]:
wandb_callback.finish()